In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1ec1169df2ee12a1ce404d055c2d657ce6ae836403e2ed4fc74de7ed889f5244
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, isnull
import sys
import json
from operator import add
import string
import csv
from itertools import combinations
import copy
import math
import time
from functools import reduce
import numpy as np 
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName("PCY").getOrCreate()
spark_df = spark.read.option("header",True).csv("./Gro_dataset.csv")

In [ ]:
spark_df.count()

38765

In [ ]:
#Missing values
missing_counts = spark_df.select([count(when(isnull(c) | isnan(c), c)).alias(c) for c in spark_df.columns])
missing_counts.show()

+-------------+----+---------------+
|Member_number|Date|itemDescription|
+-------------+----+---------------+
|            0|   0|              0|
+-------------+----+---------------+



In [ ]:
spark_df.show(5)

+-------------+----------+----------------+
|Member_number|      Date| itemDescription|
+-------------+----------+----------------+
|         1808|21-07-2015|  tropical fruit|
|         2552|05-01-2015|      whole milk|
|         2300|19-09-2015|       pip fruit|
|         1187|12-12-2015|other vegetables|
|         3037|01-02-2015|      whole milk|
+-------------+----------+----------------+
only showing top 5 rows



In [ ]:
summary1 = spark_df.summary().toPandas()
summary1

,summary,Member_number,Date,itemDescription
0,count,38765,38765,38765
1,mean,3003.64186766413,None,None
2,stddev,1153.6110310565457,None,None
3,min,1000,01-01-2014,Instant food products
4,25%,2002.0,None,None
5,50%,3005.0,None,None
6,75%,4007.0,None,None
7,max,5000,31-10-2015,zwieback


In [ ]:
# Count the unique items in a specific column
unique_items = spark_df.select(col("itemDescription")).distinct().count()

# Print the count
print("Unique item count:", unique_items)

Unique item count: 167


In [ ]:
# Group the data by the column and count the occurrences in Ascending and descending order
value_counts = spark_df.groupBy(col("itemDescription")).count()

# Sort the result in ascending order
sorted_counts = value_counts.orderBy(col("count"))

# Show the sorted result
sorted_counts.show()

# Show the result
value_counts.show()

+--------------------+-----+
|     itemDescription|count|
+--------------------+-----+
|preservation prod...|    1|
|     kitchen utensil|    1|
|      baby cosmetics|    3|
|                bags|    4|
|      toilet cleaner|    5|
|     rubbing alcohol|    5|
|      frozen chicken|    5|
|     make up remover|    5|
|      salad dressing|    6|
|              whisky|    8|
|             liqueur|    9|
|         decalcifier|    9|
|          hair spray|    9|
|    organic products|   10|
|specialty vegetables|   11|
|       frozen fruits|   11|
|               cream|   12|
|               honey|   13|
|   cooking chocolate|   15|
|         ready soups|   15|
+--------------------+-----+
only showing top 20 rows

+------------------+-----+
|   itemDescription|count|
+------------------+-----+
|         beverages|  251|
|pickled vegetables|  134|
|    snack products|   27|
|           vinegar|   51|
|    red/blush wine|  157|
|          cat food|  177|
|            dishes|  135|
|       

**We have 167 unique items from above we can assume the minsupport as 25**

In [ ]:
spark_df.show(5)

+-------------+----------+----------------+
|Member_number|      Date| itemDescription|
+-------------+----------+----------------+
|         1808|21-07-2015|  tropical fruit|
|         2552|05-01-2015|      whole milk|
|         2300|19-09-2015|       pip fruit|
|         1187|12-12-2015|other vegetables|
|         3037|01-02-2015|      whole milk|
+-------------+----------+----------------+
only showing top 5 rows



In [ ]:
# Distinct number of records
spark_df.distinct().count()

38006

In [ ]:
spark_df.count()

38765

In [ ]:
# DF of distinct items
items = spark_df.select('itemDescription').distinct()
items.show(10)

+------------------+
|   itemDescription|
+------------------+
|         beverages|
|pickled vegetables|
|    snack products|
|           vinegar|
|    red/blush wine|
|          cat food|
|            dishes|
|          cake bar|
|   organic sausage|
|           napkins|
+------------------+
only showing top 10 rows



#DATA PREPROCESSING

In [ ]:
#Grouping all the items into baskets of each member_number
from pyspark.sql import functions as F
df_grouped = spark_df.groupBy(['Member_number', 'date']).agg(F.collect_list("itemDescription"))

In [ ]:
df_grouped.show()

+-------------+----------+-----------------------------+
|Member_number|      date|collect_list(itemDescription)|
+-------------+----------+-----------------------------+
|         1000|15-03-2015|         [sausage, whole m...|
|         1000|24-06-2014|         [whole milk, past...|
|         1000|24-07-2015|         [canned beer, mis...|
|         1000|25-11-2015|         [sausage, hygiene...|
|         1000|27-05-2015|         [soda, pickled ve...|
|         1001|02-05-2015|          [frankfurter, curd]|
|         1001|07-02-2014|         [sausage, whole m...|
|         1001|12-12-2014|           [whole milk, soda]|
|         1001|14-04-2015|          [beef, white bread]|
|         1001|20-01-2015|         [frankfurter, sod...|
|         1002|09-02-2014|         [frozen vegetable...|
|         1002|26-04-2014|         [butter, whole milk]|
|         1002|26-04-2015|         [tropical fruit, ...|
|         1002|30-08-2015|         [butter milk, spe...|
|         1003|10-02-2015|     

In [ ]:
#After combining all the items to particular bill
summary2 = df_grouped.summary().toPandas()
summary2

,summary,Member_number,date
0,count,14963,14963
1,mean,3006.840807324734,None
2,stddev,1152.9474711338305,None
3,min,1000,01-01-2014
4,25%,2005.0,None
5,50%,3010.0,None
6,75%,4015.0,None
7,max,5000,31-10-2015


In [ ]:
df_grouped.printSchema()

root
 |-- Member_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- collect_list(itemDescription): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
df_grouped = df_grouped.withColumnRenamed("collect_list(itemDescription)", "items")

In [ ]:
df_grouped.printSchema()

root
 |-- Member_number: string (nullable = true)
 |-- date: string (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
df_grouped.show(10)

+-------------+----------+--------------------+
|Member_number|      date|               items|
+-------------+----------+--------------------+
|         1000|15-03-2015|[sausage, whole m...|
|         1000|24-06-2014|[whole milk, past...|
|         1000|24-07-2015|[canned beer, mis...|
|         1000|25-11-2015|[sausage, hygiene...|
|         1000|27-05-2015|[soda, pickled ve...|
|         1001|02-05-2015| [frankfurter, curd]|
|         1001|07-02-2014|[sausage, whole m...|
|         1001|12-12-2014|  [whole milk, soda]|
|         1001|14-04-2015| [beef, white bread]|
|         1001|20-01-2015|[frankfurter, sod...|
+-------------+----------+--------------------+
only showing top 10 rows



In [ ]:
print("NUmber of unique customers :")
df_grouped.select('Member_number').distinct().count()

NUmber of unique customers :


3898

In [ ]:
df_grouped.count()

14963

In [ ]:
# Saving the transactions as csv file
df_grouped.toPandas().loc[:, ["items"]].to_csv("transactions.csv")

In [ ]:
# Dropping member_number and date columns
df = df_grouped.drop("Member_number")
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
transactions_with_header = df.drop("date")
transactions_with_header.printSchema()

root
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [ ]:
#Total items 
transactions_with_header.show(5)

+--------------------+
|               items|
+--------------------+
|[sausage, whole m...|
|[whole milk, past...|
|[canned beer, mis...|
|[sausage, hygiene...|
|[soda, pickled ve...|
+--------------------+
only showing top 5 rows



In [ ]:
transactions_with_header.count()

14963

In [ ]:
transactions_with_header.toPandas().loc[:, ["items"]].to_csv('transactions.csv')

## PCY Implementation Using RDD

In [ ]:
def openfile():
    file_name = "transactions.csv"
    minsupport = 20
    nbuckets=20
    
    sc = spark.sparkContext
    data = sc.textFile(file_name)
#     data = data.map(lambda line : ''.join(line.split(',')))
    newfile = data.collect()
    
    buckets=[]
    for x in newfile:
       buckets.append([y.strip() for y in x.split(',')])

    size1freqset(minsupport,nbuckets,buckets)

In [ ]:
def size1freqset(minsupport,nbuckets,buckets):

    candidatelist1=[]
    finalist1=[]

    for i in range(0, len(buckets)):
        for b in buckets[i]:
            candidatelist1.append(b)

    candidatelist1=sorted(set(candidatelist1))
    # print(candidatelist1)
    
    lala=0
    for k in candidatelist1:
        for i in range(0, len(buckets)):
            for j in buckets[i]:
                if(k==j):
                    lala+=1
        if lala>=minsupport:
            finalist1.append(k)
        lala=0

    itemiddic={}
    counter=1
    for c in candidatelist1:
        itemiddic[c]=counter
        counter+=1

    if finalist1:
        print("Frequent Itemsets of size 1")
        with open("output1.txt", "w") as f:
            f.write("Frequent Itemsets of size 1 \n" + "Total number of itemsets : " + str(len(finalist1)) + '\n'.join(finalist1))
        for x in finalist1:
            print(x)
        size2freqset(minsupport,nbuckets,itemiddic,buckets,finalist1)
    else:
        print("That's all folks!")

In [ ]:
import sys
def size2freqset(minsupport,nbuckets,itemiddic,buckets,finalist1):

    k=3

    countofbuckets=[0]*nbuckets
    bitmap=[0]*nbuckets
    pairs=[]

    """PCY Pass 1"""
    for i in range(0,len(buckets)):
        for x in range(0,len(buckets[i])-1):
            for y in range(x+1,len(buckets[i])):
                if(buckets[i][x]<buckets[i][y]):
                    countofbuckets[int(str(itemiddic[buckets[i][x]])+str(itemiddic[buckets[i][y]]))%nbuckets]+=1
                    if ([buckets[i][x],buckets[i][y]] not in pairs):
                        pairs.append(sorted([buckets[i][x],buckets[i][y]]))
                else:
                    countofbuckets[int(str(itemiddic[buckets[i][y]])+str(itemiddic[buckets[i][x]]))%nbuckets]+=1
                    if ([buckets[i][y],buckets[i][x]] not in pairs):
                        pairs.append(sorted([buckets[i][y],buckets[i][x]]))

    pairs=sorted(pairs)

    for x in range(0,len(countofbuckets)):
        if countofbuckets[x]>=minsupport:
            bitmap[x]=1
        else:
            bitmap[x]=0

    prunedpairs=[]

    """Checking condition 1 of PCY Pass 2"""

    for i in range(0,len(pairs)):
        for j in range(0,len(pairs[i])-1):
            if(pairs[i][j] in finalist1 and pairs[i][j+1] in finalist1):
                prunedpairs.append(pairs[i])

    candidatelist2=[]

    """Checking condition 2 of PCY Pass 2"""
    for i in range(0, len(prunedpairs)):
        for j in range(0,len(prunedpairs[i])-1):
            if bitmap[int(str(itemiddic[prunedpairs[i][j]])+str(itemiddic[prunedpairs[i][j+1]]))%nbuckets]==1:
                candidatelist2.append(prunedpairs[i])

    """Appending frequent items of size 2 to finallist2"""
    finalist2=[]
    p=0
    for c in range(0,len(candidatelist2)):
        for b in range(0,len(buckets)):
            if set(candidatelist2[c]).issubset(set(buckets[b])):
                p+=1
        if p>=minsupport and p!=0:
            finalist2.append(sorted(candidatelist2[c]))
        p=0

    finalist2=sorted(finalist2)

    if finalist2:
        print("\nFrequent Itemsets of size 2")
        sys.set_int_max_str_digits(5016)
        with open("output2.txt", "w") as f:
            f.write("Frequent Itemsets of size 2 \n" + "Total number of itemsets : " + str(len(finalist1)) + '\n'.join([','.join(map(str, item)) for item in finalist2])) #(finalist2))
        for b in finalist2:
            print(','.join(b))
        sizekfreqset(minsupport,nbuckets,itemiddic,buckets,finalist2,k)
    else:
        print("End")

In [ ]:
def sizekfreqset(minsupport,nbuckets,itemiddic,buckets,prevout,k):

	"""Candidate list"""
	
	bucketcountk=[0]*nbuckets
	bitmapofsizk=[0]*nbuckets
	kcombination=[]
	prevout=prevout
	
	#k combinations
	for a in prevout:
		for b in prevout:
			if(a!=b):
				if set(a) & set(b) and len(list(set(a) & set(b))) >= k-2:
					kcombination.append(sorted(set(a)|set(b)))

	# print("checking combination of size k")
	kcombination=sorted(kcombination)

	"""PCY Pass 1 Hashing"""

	hashingstring=""

	for i in range(0,len(kcombination)):
		for x in kcombination[i]:
			hashingstring+=str(itemiddic[x])
		bucketcountk[int(hashingstring)%nbuckets]+=1
		hashingstring=""

	for x in range(0,len(bucketcountk)):
		if bucketcountk[x]>=minsupport:
			bitmapofsizk[x]=1
		else:
			bitmapofsizk[x]=0

	"""Condition 1 is automatically satisfied"""

	"""Checking condition 2 of PCY Pass 2"""
	hashingstring=""
	klist=[]
	
	for i in range(0, len(kcombination)):
		for j in kcombination[i]:
			hashingstring+=str(itemiddic[j])
		if bitmapofsizk[int(hashingstring)%nbuckets]==1:
			klist.append(kcombination[i])
		teststring=""

	"""Creating Candidate List of Size k"""
	candidatelistk=[]
	count=1
	for i in range(1, len(klist)):
		if klist[i]==klist[i-1]:
			count+=1
		else:
			# print("ELement is: ",klist[i-1]," and count is: ",count)
			if count>=k:
				candidatelistk.append(klist[i-1])
			count=1
	if count>=k:
		candidatelistk.append(klist[i-1])
		# print("ELement is: ",klist[i-1]," and count is: ",count)
	
	"""Appending frequent items of size k to output"""
	output=[]
	counter=0
	for c in range(0,len(candidatelistk)):
		for b in range(0,len(buckets)):
			if set(candidatelistk[c]).issubset(set(buckets[b])):
				counter+=1
		if counter>=minsupport and counter!=0:
			output.append(candidatelistk[c])
		counter=0

	if output:
		print("\nFrequent Itemsets of size ",k)
		for b in output:
			print(','.join(b))
		k+=1
		sizekfreqset(minsupport,nbuckets,itemiddic,buckets,output,k)
		# print("\n")
	else:
		print("End")

In [ ]:
openfile()

Frequent Itemsets of size 1
"['UHT-milk'
"['baking powder'
"['beef'
"['berries'
"['beverages'
"['bottled beer'
"['bottled water'
"['brown bread'
"['butter milk'
"['butter'
"['candy'
"['canned beer'
"['canned fish'
"['cat food'
"['chewing gum'
"['chicken'
"['chocolate'
"['citrus fruit'
"['coffee'
"['condensed milk'
"['cream cheese '
"['curd'
"['dessert'
"['detergent'
"['dishes'
"['domestic eggs'
"['finished products'
"['fish'
"['frankfurter'
"['frozen dessert'
"['frozen fish'
"['frozen meals'
"['frozen vegetables'
"['fruit/vegetable juice'
"['grapes'
"['ham'
"['hamburger meat'
"['hard cheese'
"['herbs'
"['hygiene articles'
"['ice cream'
"['liquor'
"['liver loaf'
"['long life bakery product'
"['margarine'
"['meat'
"['misc. beverages'
"['napkins'
"['newspapers'
"['oil'
"['onions'
"['other vegetables'
"['packaged fruit/vegetables'
"['pasta'
"['pastry'
"['pet care'
"['photo/film'
"['pickled vegetables'
"['pip fruit'
"['pork'
"['pot plants'
"['processed cheese'
"['red/blush wine'
"['rolls/bu

ValueError: ignored